# Tabular playground Series 2021 ✨ 

#### **Task: Regression problem** 😊 

Previously try : [https://www.kaggle.com/venkatkumar001/tps-sep-21-day2-eda-xgb](http://)

**Version 4 Newly try voting method of regression problem**

#### **Process** ✌️ 

1. Read the data
2. Identify the null data and apply simple imputer(simple imputer identify the missing data and fill mean values)
3. Preprocessing the train and test data
4. Apply Standardscaler and xgb,catboost,Lgbm
5. Submitting the output files

# Import Necessary Library ✨ ✌️ 

In [ ]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams["figure.figsize"] = (12,10)
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

#Model library
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor


# Read the data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
sample= pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
train.shape,test.shape

#  Identify the null data and drop

In [ ]:
train.isnull().sum()

In [ ]:
train.describe(include='all')

# KFold

In [ ]:
#insert the kfold columns
train['kfold'] = -1
#distributing the data
kfold = KFold(n_splits = 5,shuffle=True,random_state = 42)
for fold, (tr_i,va_i) in enumerate(kfold.split(X=train)):
    train.loc[va_i,'kfold'] = fold
    
print(train.kfold.value_counts())
train.to_csv("folds_5.csv",index=False)
print("successfully folds")


In [ ]:
train=train.dropna(axis=0)
train.isnull().sum()

In [ ]:
train.describe(include='all')

# Preprocessing the data (StandardScaler)

# XGBOOSTREGRESSOR

In [ ]:
#Folds data
train = pd.read_csv("./folds_5.csv")

#features taken to train
features = [f for f in train.columns if f not in("id","kfold","claim")]
num_cols = [cols for cols in features if 'f' in cols]

test= test[features]
prediction = []
score = []

for fold in range (5):
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    lE = StandardScaler()
    xtrain[num_cols] = lE.fit_transform(xtrain[num_cols])
    xvalid[num_cols] = lE.transform(xvalid[num_cols])
    xtest[num_cols] = lE.transform(xtest[num_cols])
    
    
    
    #xgboost model
    xgb_m = XGBRegressor(learning_rate=0.021537077920105466,
                         n_estimators=10606,
                         random_state=228,
                         gpu_id=0,
                         tree_method='gpu_hist',
                         predictor='gpu_predictor'
                        )
    xgb_m.fit(xtrain,ytrain,early_stopping_rounds=100,eval_set=[(xvalid,yvalid)],verbose=1000)
    predict_valid = xgb_m.predict(xvalid)
    test_predict = xgb_m.predict(xtest)
    prediction.append(test_predict)
    
    #Root_mean_square
    rms = mean_squared_error(yvalid,predict_valid,squared=False)
    
    #Score 
    score.append(rms)
    print(f"fold|split:{fold},rmse:{rms}")
    
print(np.mean(score),np.std(score))




In [ ]:
#reconfigure of split data
final_predict = np.mean(np.column_stack(prediction),axis=1)
print(final_predict)
sample.claim = final_predict
sample.to_csv("First_submission_xgb.csv",index=False)
print("Final achieve to send xgboost output data")

# CATBOOSTREGRESSOR ✨ 

In [ ]:
#Folds data
train = pd.read_csv("./folds_5.csv")

#features taken to train
features = [f for f in train.columns if f not in("id","kfold","claim")]
num_cols = [cols for cols in features if 'f' in cols]

test= test[features]

prediction0 = []
score = []

for fold in range (5):
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    lE = StandardScaler()
    xtrain[num_cols] = lE.fit_transform(xtrain[num_cols])
    xvalid[num_cols] = lE.transform(xvalid[num_cols])
    xtest[num_cols] = lE.transform(xtest[num_cols])
    
    #catboost model
    catpara={
        'subsample': 0.95312,
        'learning_rate': 0.014589235,
        "max_depth": 6,
        "min_data_in_leaf":77,
        'random_state':228,
        'n_estimators':10000,
        'rsm':0.5,
        'l2_leaf_reg': 0.02247766515106271
    }
    
    model=CatBoostRegressor(**catpara)
    model.fit(xtrain,ytrain,early_stopping_rounds=100,eval_set=[(xvalid,yvalid)],verbose=1000)
    
    predict_valid = model.predict(xvalid)
    test_predict1 = model.predict(xtest)
    prediction0.append(test_predict1)
    
    #Root_mean_square
    rms = mean_squared_error(yvalid,predict_valid,squared=False)
    
    #Score 
    score.append(rms)
    print(f"fold|split:{fold},rmse:{rms}")
    
print(np.mean(score),np.std(score))

In [ ]:
#reconfigure of split data
pred0 = np.mean(np.column_stack(prediction0),axis=1)
print(pred0)
sample.claim = pred0
sample.to_csv("mysubmission_cat.csv",index=False)
print("Finally send the output data")

# LIGHTGBM ✨ 

In [ ]:
#Folds data
train = pd.read_csv("./folds_5.csv")

#features taken to train
features = [f for f in train.columns if f not in("id","kfold","claim")]
num_cols = [cols for cols in features if 'f' in cols]

test= test[features]

prediction1 = []
score = []

for fold in range (5):
    xtrain = train[train.kfold != fold].reset_index(drop=True)
    xvalid = train[train.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.claim
    yvalid = xvalid.claim
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    lE = StandardScaler()
    xtrain[num_cols] = lE.fit_transform(xtrain[num_cols])
    xvalid[num_cols] = lE.transform(xvalid[num_cols])
    xtest[num_cols] = lE.transform(xtest[num_cols])
    
    #lgb parameters
    params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "regression",
    'subsample': 0.95312,
    "metric": "rmse",
    'learning_rate': 0.01635,
    "max_depth": 3,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.7705303688019942,
    "min_child_samples": 290,
    "reg_alpha": 14.68267919457715,
    "reg_lambda": 66.156,
    "max_bin": 772,
    "min_data_per_group": 177,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 17,
    "verbosity": -1,
    'random_state':288,
    'n_estimators':11990,
    'colsample_bytree':0.1107,
    'njobs':4
    }
    
    lgb_train = lgb.Dataset(xtrain, ytrain)
    lgb_val = lgb.Dataset(xvalid, yvalid)
    
    model = lgb.train(params=params_lgb,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      early_stopping_rounds=300,
                      verbose_eval=1000)
    
   
    preds_valid = model.predict(xvalid,num_iteration=model.best_iteration)
    test_predict = model.predict(xtest,num_iteration=model.best_iteration)
    
    prediction1.append(test_predict)
    
    #Root_mean_square
    rms = mean_squared_error(yvalid,preds_valid,squared=False)
    
    #Score 
    score.append(rms)
    print(f"fold|split:{fold},rmse:{rms}")
    
print(np.mean(score),np.std(score))

## Submission File


In [ ]:
#reconfigure of split data
pred1 = np.mean(np.column_stack(prediction1),axis=1)
print(pred1)
sample.claim = pred1
sample.to_csv("mysubmission_lgb.csv",index=False)
print("Finally send the output data")

In [ ]:
print("Finally achieve to send  output data")

## **Thankyou** ✨ ✌️ 👍 😊 